In [1]:
from __future__ import print_function

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.utils import to_categorical
import numpy as np

import jieba
from nltk import word_tokenize

Using TensorFlow backend.


## read data

In [2]:
batch_size = 64  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
EMBEDDING_DIM = 100

# Path to the data txt file on disk.
data_path = 'cmn.txt'

In [3]:
# Vectorize the data.
input_texts_original = []
input_texts = []
target_texts = []
input_words = set()
target_words = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    print('sample size:', len(lines))

for line in lines[:10000]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
        
    input_texts_original.append(input_text)
    input_text = word_tokenize(input_text)
    input_text = [word.lower() for word in input_text]
    target_text = list(jieba.cut(target_text))

    for word in input_text:
        if word not in input_words:
            input_words.add(word)
    for word in target_text:
        if word not in target_words:
            target_words.add(word)
            
    target_text = ['\t'] + target_text + ['\n']
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
print('input vocalbulary size:', len(input_words))
print('target vocalbulary size:', len(target_words))

# del lines, line, input_text, target_text, word

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\溫子漢\AppData\Local\Temp\jieba.cache


sample size: 20295


Loading model cost 1.313 seconds.
Prefix dict has been built succesfully.


input vocalbulary size: 3521
target vocalbulary size: 6993


## 準備pretrained vectors

In [4]:
input_tokens = ['<PAD>', '<UNK>']
input_embedding_matrix = [np.zeros([EMBEDDING_DIM])]

with open('glove.6B.100d.txt', encoding='utf8') as f:
    for i, line in enumerate(f):
        values = line.split()
        word = values[0]
        if word == 'unk':
            print('unk at line %s in file' % i)
            coefs = np.asarray(values[1:], dtype='float32')
            input_embedding_matrix.insert(1, coefs)
        elif word in input_words:
            coefs = np.asarray(values[1:], dtype='float32')
            input_tokens.append(word)
            input_embedding_matrix.append(coefs)
            
input_embedding_matrix = np.vstack(input_embedding_matrix)

print('Found %s word vectors.' % len(input_tokens))

# del coefs, values, word, input_words, line

unk at line 201534 in file
Found 3518 word vectors.


In [5]:
target_words = sorted(list(target_words))
target_words = ['<PAD>', '\t', '\n'] + target_words

num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 3518
Number of unique output tokens: 6996
Max sequence length for inputs: 12
Max sequence length for outputs: 15


## 準備 input array

In [6]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_tokens)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, word in enumerate(input_text):
        embedding_index = input_token_index.get(word)
        if embedding_index is not None:
            encoder_input_data[i, t] = embedding_index
        else:
            encoder_input_data[i, t] = input_token_index['<UNK>']
            
    for t, word in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        if word != '\n':
            decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1] = target_token_index[word]
            
decoder_target_data = to_categorical(decoder_target_data, num_classes=num_decoder_tokens)
            
# del input_tokens, target_words

## define model

In [8]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=[max_encoder_seq_length])

encoder_embedding = Embedding(num_encoder_tokens,
                            EMBEDDING_DIM,
                            weights=[input_embedding_matrix],
#                             input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)(encoder_inputs)

encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embedding)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=[max_decoder_seq_length])

embedding = Embedding(num_decoder_tokens,
                            EMBEDDING_DIM,
#                             weights=[input_embedding_matrix],
#                             input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
decoder_embedding = embedding(decoder_inputs)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s-epoch.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 42s 5ms/step - loss: 2.5637 - val_loss: 2.6475
Epoch 2/50
8000/8000 [==============================] - 33s 4ms/step - loss: 1.9214 - val_loss: 2.4842
Epoch 3/50
8000/8000 [==============================] - 35s 4ms/step - loss: 1.7674 - val_loss: 2.4380
Epoch 4/50
8000/8000 [==============================] - 37s 5ms/step - loss: 1.6570 - val_loss: 2.3059
Epoch 5/50
4416/8000 [===============>..............] - ETA: 14s - loss: 1.5619

In [25]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding = embedding(decoder_inputs)

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [26]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [27]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, max_decoder_seq_length))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['\t']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    i = 1
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, i-1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char + ' '#+'('+str(sampled_token_index)+')'

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or i >= max_decoder_seq_length-1): #max_decoder_seq_length
            stop_condition = True

        # Update the target sequence (of length 1).
#         target_seq = np.zeros((1, max_decoder_seq_length))
        target_seq[0, i] = sampled_token_index

        # Update states
#         states_value = [h, c]
        
        i = i + 1

    return decoded_sentence

In [29]:
req_index = [4077, 2122, 3335, 1464, 8956,
             7168, 3490, 4495, 5100, 119]

output_string = ''
for seq_index in list(range(25)):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    
    output_string += input_texts_original[seq_index] + '\n'
    output_string += decoded_sentence

-
Input sentence: ['hi', '.']
Decoded sentence: 嗨 。 
 
-
Input sentence: ['hi', '.']
Decoded sentence: 嗨 。 
 
-
Input sentence: ['run', '.']
Decoded sentence: 你 用 跑 的 。 
 
-
Input sentence: ['wait', '!']
Decoded sentence: 等等 ！ 
 
-
Input sentence: ['hello', '!']
Decoded sentence: 你好 。 
 
-
Input sentence: ['i', 'try', '.']
Decoded sentence: 让 我 来 。 
 
-
Input sentence: ['i', 'won', '!']
Decoded sentence: 我 赢 了 。 
 
-
Input sentence: ['oh', 'no', '!']
Decoded sentence: 不会 吧 。 
 
-
Input sentence: ['cheers', '!']
Decoded sentence: 乾杯 ! 
 
-
Input sentence: ['he', 'ran', '.']
Decoded sentence: 他 跑 了 。 
 
-
Input sentence: ['hop', 'in', '.']
Decoded sentence: 跳进来 。 
 
-
Input sentence: ['i', 'lost', '.']
Decoded sentence: 我 迷失 了 。 
 
-
Input sentence: ['i', 'quit', '.']
Decoded sentence: 我 退出 。 
 
-
Input sentence: ['i', "'m", 'ok', '.']
Decoded sentence: 我 沒事 。 
 
-
Input sentence: ['listen', '.']
Decoded sentence: 听 着 。 
 
-
Input sentence: ['no', 'way', '!']
Decoded sentence: 没门 ！ 
 
-


In [25]:
with open('output.txt', 'w') as f:
    f.write(output_string)